In [4]:
import pandas as pd
import statsmodels.api as sm
import sys
sys.path.append(r'C:\Users\camde\OneDrive - Brigham Young University\IS JUNIOR CORE-Camdens-Laptop\IS 455\myfunctions')
import myfunctions as my

df = pd.read_csv(r'C:\Users\camde\OneDrive - Brigham Young University\IS JUNIOR CORE-Camdens-Laptop\IS 455\univariate_automation\insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [7]:
y = df['charges']
X = pd.get_dummies(df.drop(columns=['charges']), drop_first=True)
X[X.select_dtypes(bool).columns] = X.select_dtypes(bool).astype(int)

model = sm.OLS(y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                charges   R-squared (uncentered):                   0.874
Model:                            OLS   Adj. R-squared (uncentered):              0.874
Method:                 Least Squares   F-statistic:                              1158.
Date:                Thu, 20 Feb 2025   Prob (F-statistic):                        0.00
Time:                        12:52:33   Log-Likelihood:                         -13618.
No. Observations:                1338   AIC:                                  2.725e+04
Df Residuals:                    1330   BIC:                                  2.729e+04
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

In [ ]:
df_results = pd.DataFrame({'Predicted': model.fittedvalues,
                           'Actual' : y,
                           'Difference' : abs(y - model.fittedvalues),
                           'Squared diff' : (y - model.fittedvalues)**2})

mae = df_results['Difference'].mean()
print(f"Mean absolute error (MAE):\t${mae:.2f}")

rmse = df_results['Squared diff'].mean()**(1/2)
print(f"Root mean squared error (RMSE):\t${rmse:.2f}")

df_results.head()

Mean absolute error (MAE):	$4367.97
Root mean squared error (RMSE):	$6365.04


,Predicted,Actual,Difference,Squared diff
0,27214.565779,16884.92400,10329.641779,1.067015e+08
1,4335.391396,1725.55230,2609.839096,6.811260e+06
2,6890.166546,4449.46200,2440.704546,5.957039e+06
3,5870.513524,21984.47061,16113.957086,2.596596e+08
4,6095.351356,3866.85520,2228.496156,4.966195e+06


In [ ]:
import numpy as np

# fix charges skewness for a better model
y = np.log(df['charges'])
X = pd.get_dummies(df.drop(columns=['charges']), drop_first=True)
X[X.select_dtypes(bool).columns] = X.select_dtypes(bool).astype(int)

model = sm.OLS(y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                charges   R-squared (uncentered):                   0.981
Model:                            OLS   Adj. R-squared (uncentered):              0.981
Method:                 Least Squares   F-statistic:                              8593.
Date:                Thu, 20 Feb 2025   Prob (F-statistic):                        0.00
Time:                        13:13:56   Log-Likelihood:                         -2207.7
No. Observations:                1338   AIC:                                      4431.
Df Residuals:                    1330   BIC:                                      4473.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

In [25]:
df_results = pd.DataFrame({'Predicted': np.exp(model.fittedvalues),
                           'Actual' : np.exp(y),
                           'Difference' : abs(np.exp(y) - np.exp(model.fittedvalues)),
                           'Squared diff' : (np.exp(y) - np.exp(model.fittedvalues))**2})

mae = df_results['Difference'].mean()
print(f"Mean absolute error (MAE):\t${mae:.2f}")

rmse = df_results['Squared diff'].mean()**(1/2)
print(f"Root mean squared error (RMSE):\t${rmse:.2f}")

df_results.head()

Mean absolute error (MAE):	$23469.42
Root mean squared error (RMSE):	$71333.29


,Predicted,Actual,Difference,Squared diff
0,4251.192452,16884.92400,12633.731548,1.596112e+08
1,1722.811397,1725.55230,2.740903,7.512550e+00
2,4470.688480,4449.46200,21.226480,4.505635e+02
3,1200.275048,21984.47061,20784.195562,4.319828e+08
4,3255.497525,3866.85520,611.357675,3.737582e+05


In [27]:
# code from ch. 11 multicollinearity
import statsmodels.api as sm

# Create a blank DataFrame to store the results
df_vif = pd.DataFrame(columns=['VIF'])

# Loop through the X features only to generate VIF score for each
for col in X:
    y = X[col] # Each X feature takes a turn being the y
    # All remaining X features are used to predict that y
    X = X.drop(columns=[col]).assign(const=1)

    r_squared = sm.OLS(y, X).fit().rsquared # Record the R squared from the model
    df_vif.loc[col] = [1/(1 - r_squared)]

# Print out the list of VIF scores sorted from highest (worst) to lowest (best)
df_vif.sort_values(by=['VIF'], ascending=False)

,VIF
region_northwest,1.516537
region_southeast,1.136234
bmi,1.091546
age,1.016822
sex_male,1.006003
smoker_yes,1.005519
children,1.002007
region_southwest,1.000000


In [28]:
df_vif.iloc[0].index[0]

'VIF'

In [31]:
'''example of how to automate dropping columns based on highest VIF value/P value'''
# while len(df_vif) <= 5:
    #train the model
    # VIF for loop to build df_vif table
    # remove from df_vif the index at the top of vif table from the original model

'example of how to automate dropping columns based on highest VIF value/P value'

In [ ]:
model.pvalues.sort_values(ascending=False)

region_southeast     9.055443e-01
sex_male             5.220779e-04
region_southwest     4.088449e-06
children             2.162855e-13
region_northwest     1.344850e-14
smoker_yes           1.912236e-87
age                 2.355066e-142
bmi                 2.525974e-279
dtype: float64